In [1]:
# creating path so jupyter notebook knows where to look for the crusher module that i downloaded
import sys
sys.path.append('/Users/sofia/Research/Work/Project 1/Modules/crusher-master')

# basic imports
import h5py
import numpy as np

import os
import warnings

import multiprocessing as mp
from scipy.interpolate import interp1d
from scipy import signal

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import matplotlib.patches as mpatches
from matplotlib.ticker import FormatStrFormatter
import matplotlib.backends.backend_pdf

import numpy as np
from astropy.io import fits

import pandas as pd

import h5py
import math

import random

import time


# import my code crusher
import crusher
from crusher.galaxy import GalaxyMap
from crusher.data import BeneMassAgeZMaps
from crusher import visual
from crusher import combine
from crusher import profile

# import benedikt code, colossus
from colossus.halo import mass_so
from colossus.cosmology import cosmology

# I get a latex error, so this fixes it
#matplotlib.rcParams['text.usetex']=False
#matplotlib.rcParams['text.latex.unicode']=False

# make axis label sizes bigger
plt.rcParams['xtick.labelsize']=24
plt.rcParams['ytick.labelsize']=24

In [2]:
# list of files
file_list_lr_100 = ['galaxies_progmaps_lr_tng100_099.hdf5',
                    'galaxies_progmaps_lr_tng100_084.hdf5', 
                    'galaxies_progmaps_lr_tng100_072.hdf5',
                    'galaxies_progmaps_lr_tng100_067.hdf5', 
                    'galaxies_progmaps_lr_tng100_059.hdf5',
                    'galaxies_progmaps_lr_tng100_050.hdf5',
                    'galaxies_progmaps_lr_tng100_040.hdf5',
                    'galaxies_progmaps_lr_tng100_033.hdf5']

file_list_lr_300 = ['galaxies_progmaps_lr_tng300_099.hdf5',
                    'galaxies_progmaps_lr_tng300_084.hdf5', 
                    'galaxies_progmaps_lr_tng300_072.hdf5',
                    'galaxies_progmaps_lr_tng300_067.hdf5', 
                    'galaxies_progmaps_lr_tng300_059.hdf5',
                    'galaxies_progmaps_lr_tng300_050.hdf5',
                    'galaxies_progmaps_lr_tng300_040.hdf5']

file_list_hr_300 = ['galaxies_progmaps_hr_tng300_099.hdf5',
                    'galaxies_progmaps_hr_tng300_084.hdf5', 
                    'galaxies_progmaps_hr_tng300_072.hdf5',
                    'galaxies_progmaps_hr_tng300_067.hdf5', 
                    'galaxies_progmaps_hr_tng300_059.hdf5',
                    'galaxies_progmaps_hr_tng300_050.hdf5',
                    'galaxies_progmaps_hr_tng300_040.hdf5']

file_path_100 = '/users/sofiafranck/Data/TNG100/'
file_path_300 = '/users/sofiafranck/Data/TNG300/'

file_list = file_list_lr_300
file_path = file_path_300

form to look at a certain galaxy at a certain snapshot: np.array(tng_data_z['tree_*'])[galaxy_index, snapnum])

# Constants and random data that I might as well grab now:

In [3]:
# value for h with h0 = 70 km s^-1 Mpc^-1
h = 0.7
box_size = 205 * 1000 #kpc

# some data sets at z=0 and some of their fields
tng_data_099 = h5py.File(file_path + file_list[0], 'r')
tng_bene_099 = BeneMassAgeZMaps(file_path + file_list[0], label='tng300_hr')

# basic info for z = 0 snapshot
is_primary_099 = np.array(tng_data_099['catgrp_is_primary'])
subhalo_mass_099 = np.array(tng_data_099['catsh_SubhaloMassType'])

# grab needed trees
tree_Group_M_TopHat200 = np.array(tng_data_099['tree_Group_M_TopHat200'])
tree_SnapNum = np.array(tng_data_099['tree_SnapNum'])
tree_SubhaloCM = np.array(tng_data_099['tree_SubhaloCM'])
tree_SubhaloMassType = np.array(tng_data_099['tree_SubhaloMassType'])
tree_SubhaloPos = np.array(tng_data_099['tree_SubhaloPos'])
tree_SubhaloSFR = np.array(tng_data_099['tree_SubhaloSFR'])
tree_SubhaloVel = np.array(tng_data_099['tree_SubhaloVel'])
tree_SubhaloVelDisp = np.array(tng_data_099['tree_SubhaloVelDisp'])
tree_SubhaloVmax = np.array(tng_data_099['tree_SubhaloVmax'])
tree_is_primary = np.array(tng_data_099['tree_is_primary'])
tree_subfind_id = np.array(tng_data_099['tree_subfind_id'])

print(tree_SnapNum, tree_SubhaloMassType)

# redshift and time lists
redshifts = np.array(tng_data_099['info']['tree_z'])
times = np.array(tng_data_099['info']['tree_t'])
snaps = np.array(tng_data_099['info']['tree_snaps'])

# redshifts and time lists at the snapshots
snapshots = [99, 84, 72, 67, 59, 50, 40]
snap_times = times[snapshots]
snap_z = redshifts[snapshots]

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/users/sofiafranck/Data/TNG300/galaxies_progmaps_lr_tng300_099.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Some functions to grab data from tng data:

In [ ]:
# function to create galaxy map from crusher and find m100
def get_masses(tng_data, i, m10_add=True, m100_add=True, proj='xy'):
    gal = GalaxyMap(tng_data, i ,proj=proj, aper_force=None)

    # m10 and m100
    r10, m10 = profile.aperture_masses(gal.info, gal.maps['mass_gal'], rad=100)
    r100, m100 = profile.aperture_masses(gal.info, gal.maps['mass_gal'], rad=100)

    # add to list what want to return
    return_list = list()
    if m10_add == True:
        return_list.append(m10)
        
    if m100_add == True:
        return_list.append(m100)
        
    return return_list

# some functions to look at tng data across snapshots

# function from joe to find ids of galaxies in different snapshot
def find_id(hdf0, hdf1, snapnum):
    ids = hdf0['tree_subfind_id'][:,snapnum]
    idx = hdf1['catsh_id'][:].argsort()

    match = hdf1['catsh_id'][:].searchsorted(ids, sorter=idx)
    goodmatch = hdf1['catsh_id'][:][idx][match]==ids

    # print(np.isclose(hdf0['tree_SubhaloMassType'][:,snapnum,4][i], hdf1['scalar_star_mass'][:][idx][match][i]))
    
    return match, goodmatch, idx

# function to find catgrp_id for any snapshot
def find_catgrp_id(tng_data_z, match, idx, high_z):
    # grab positions and convert
    if high_z == False:
        tng_grp_id = np.array(tng_data_z['catgrp_id'])
        
    if high_z == True:
        tng_grp_id = np.array(tng_data_z['catgrp_id'])[idx]

    return tng_grp_id

# function to find 3d masses for any snapshot
# kind of unfortunate to have separate data for insitu, exsitu, other subhalo (oshs) and fuzz,
# so i need to grab them all separately. looking back, i suppose i couldve used a pandas dataframe,
# but i don't think thats not a big issue 
def find_3d_masses(tng_data_z, match, idx, sat, high_z):
    if high_z == False:
        tng_volume = np.array(tng/q_data_z['profile_bins_volume'])
        tng_bins = np.array(tng_data_z['profile_bins'])
        
        tng_bound = np.array(tng_data_z['profile_star_rho_3d'])
        tng_bound_ins = np.array(tng_data_z['profile_star_rho_insitu_3d'])
        tng_bound_exs = np.array(tng_data_z['profile_star_rho_exsitu_3d'])
        
        tng_oshs = np.array(tng_data_z['profile_star_rho_oshs_3d'])
        tng_fuzz = np.array(tng_data_z['profile_star_rho_fuzz_3d'])

        # make sat specific
        volume = tng_volume[match][sat]
        bins = tng_bins[match][sat]
        
        bound = tng_bound[match][sat]
        bound_ins = tng_bound_ins[match][sat]
        bound_exs = tng_bound_exs[match][sat]
        
        oshs = tng_oshs[match][sat]
        fuzz = tng_fuzz[match][sat]

    if high_z == True:
        tng_volume = np.array(tng_data_z['profile_bins_volume'])[idx]
        tng_bins = np.array(tng_data_z['profile_bins'])[idx]
        
        tng_bound = np.array(tng_data_z['profile_star_rho_3d'])[idx]
        tng_bound_ins = np.array(tng_data_z['profile_star_rho_insitu_3d'])[idx]
        tng_bound_exs = np.array(tng_data_z['profile_star_rho_exsitu_3d'])[idx]
        
        tng_oshs = np.array(tng_data_z['profile_star_rho_oshs_3d'])[idx]
        tng_fuzz = np.array(tng_data_z['profile_star_rho_fuzz_3d'])[idx]

        # make sat specific
        volume = tng_volume[match[sat]]
        bins = tng_bins[match[sat]]
        
        bound = tng_bound[match[sat]]
        bound_ins = tng_bound_ins[match[sat]]
        bound_exs = tng_bound_exs[match[sat]]
        
        oshs = tng_oshs[match[sat]]
        fuzz = tng_fuzz[match[sat]]
    
    return bound, bound_ins, bound_exs, oshs, fuzz, volume, bins
    
# function to find primary across z
def find_primary(match, idx, sat, high_z):
    if high_z == False:
        is_primary = np.array(tng_data_z['catgrp_is_primary'])

        primary = is_primary[match][sat]

    if high_z == True:
        is_primary = np.array(tng_data_z['catgrp_is_primary'])[idx]

        primary = is_primary[match[sat]]

    if primary == 1:
        return False
    else:
        return True
    
# function to find star formation rate (should increase mass of satellite, or lessen visual effects of stripping)
def find_star_formation(tng_data_z, match, idx, sat, high_z):
    if high_z == False:
        tng_sf = np.array(tng_data_z['catsh_SubhaloSFR'])

        sat_sf = tng_sf[match][sat]

    if high_z == True:
        tng_sf = np.array(tng_data_z['catsh_SubhaloSFR'])[idx]

        sat_sf = tng_sf[match[sat]]

    return sat_sf

# function to find distances between satellite and central
def find_distances(tng_data, match, idx, sat, central, redshift, high_z):
    # value for h with h0 = 70 km s^-1 Mpc^-1
    h = 0.7

    if high_z == False:
        tng_cm = np.array(tng_data['catsh_SubhaloCM'])

        sat_cm = tng_cm[match][sat]
        central_cm = tng_cm[match][central]

    if high_z == True:
        tng_cm = np.array(tng_data['catsh_SubhaloCM'])[idx]

        sat_cm = tng_cm[match[sat]]
        central_cm = tng_cm[match[central]]

    # now to find the distance between the satellite and the central
    # convert from comoving coords to kpc
    sat_cm_converted = sat_cm / ((1 + redshift) * h)
    central_cm_converted = central_cm / ((1 + redshift) * h)

    #find distance -- bc sat is bound to central, must be close in z axis. so just find projection distance instead of 3d distance
    distance = math.sqrt(((sat_cm_converted[0] - central_cm_converted[0]) ** 2) + ((sat_cm_converted[1] - central_cm_converted[1]) ** 2))

    return distance
    
# misc functions

# function to find norm of array
def find_norm_array(array): 
    norm_factor = np.nanmax(array) - np.nanmin(array)

    array_norm = list()
    for i in array:
        i_norm = (i - np.nanmin(array)) / norm_factor

        array_norm.append(i_norm)
        
    return array_norm

# function to linearly interpolate the data
def interpolate(profile, rad, start, end):
    r = np.linspace(start, end, 700)
    
    interp = interp1d(rad,
                      profile,
                      bounds_error=False,
                      kind='slinear')

    # finish interpolating
    interp_map = interp(r)

    return interp_map

# function to convert comoving coords to distance
def convert_dist(sat_cm, central_cm):
    # separate center of masses into x,y,z and subtract sat and cent. also do boundary conditions
    dist_x = (sat_cm[:,0] - central_cm[:,0])
    dist_y = (sat_cm[:,1] - central_cm[:,1])
    dist_z = (sat_cm[:,2] - central_cm[:,2])

    # apply edge cases (one end of the box is connected with the other)
    # this took a little bit to figure out.
    dist_x = np.array([(box_size - abs(i)) if abs(i) > (box_size / 2) else i for i in dist_x])
    dist_y = np.array([(box_size - abs(i)) if abs(i) > (box_size / 2) else i for i in dist_y])
    dist_z = np.array([(box_size - abs(i)) if abs(i) > (box_size / 2) else i for i in dist_z])
    
    # convert distances from comoving into physical
    dist_x = np.array([(i  / ((1 + redshifts[list(dist_x).index(i)]) * h)) for i in dist_x])
    dist_y = np.array([(i  / ((1 + redshifts[list(dist_y).index(i)]) * h)) for i in dist_y])
    dist_z = np.array([(i  / ((1 + redshifts[list(dist_z).index(i)]) * h)) for i in dist_z])

    # combine into a single distance number
    distance = np.sqrt((dist_x ** 2) + (dist_y ** 2) + (dist_z ** 2))
    
    return distance

# Function to get the distances, etc for a specific satellite (main data grabbing function):

In [ ]:
# note that im pulling variables from outside the function because its just quicker

def find_data(sat, central, start_index, start, end_index, end):
    box_size = 205 * 1000 #kpc
    
    # grab necessary info from trees
    sat_sn = tree_SnapNum[sat, :]

    sat_cm = tree_SubhaloCM[sat, :]
    central_cm = tree_SubhaloCM[central, :]

    sat_mass = tree_SubhaloMassType[sat, :]
    stellar_mass = sat_mass[:,4]
    
    central_mvir = tree_Group_M_TopHat200[central, :]
    
    sat_sfr = tree_SubhaloSFR[sat, :]

    sat_is_primary = tree_is_primary[sat, :]

    # need to remove any points with -inf masses and the first instance of lists (not accurate)
    indices_mass = np.where(stellar_mass == -np.inf)
    indices_pos = np.append(np.where(sat_cm == [0,0,0]), np.where(central_cm == [0,0,0]))

    indices = np.append(indices_mass, indices_pos)
    full_indices = np.append(np.array([0,1]), indices)
    
    # remove these points with -inf masses from other lists as well
    full_indices = list(dict.fromkeys(full_indices))
    
    sat_sn = np.delete(sat_sn, full_indices, axis=0)
    sat_cm = np.delete(sat_cm, full_indices, axis=0)
    central_cm = np.delete(central_cm, full_indices, axis=0)
    sat_mass = np.delete(sat_mass, full_indices, axis=0)
    central_mvir = np.delete(central_mvir, full_indices, axis=0)
    sat_sfr = np.delete(sat_sfr, full_indices, axis=0)
    sat_is_primary = np.delete(sat_is_primary, full_indices, axis=0)

    times_o = np.delete(times, full_indices, axis=0)
    
    # just grabbing the stellar mass from the array of masses
    stellar_mass = sat_mass[:,4]
    
    # convert central of masses into distance
    distance = convert_dist(sat_cm, central_cm)
    
    # convert Mvir (virial mass) to Rvir (virial radius) using Colossus
    cosmology.setCosmology('planck18')
    central_rvir = np.array([mass_so.M_to_R(i, redshifts[list(central_mvir).index(i)], '200m') for i in central_mvir])
    
    # find snap of 2*rvir and sat mass at 2*rvir
    ratio = distance / central_rvir

    # find the first time the distance between the satellite and the central is less than 2 * central rvir
    count = 0
    prev = 0
    snap = None
    for i in ratio:
        # distance check
        if prev > 2 and i < 2 and snap is None:
            snap = count

        prev = i    
        count += 1
    
    if snap is not None:
        under_mass = sat_mass[snap, 4]
        under_sfr = sat_sfr[snap]
    
        time_under = list(tng_data_099['info']['tree_t'])[snap]
        
    else:
        print('No snap.')
    
    # error check
    if [0,0,0] in sat_cm or [0,0,0] in central_cm:
        print('[0,0,0] was found in either the sat\'s or central\'s center of mass. Likely an error.')

    # change masses into an array and normalize it for coloring in future plot
    # (pyplot colormap for viridis only takes values between 0 - 1)
    
    # also need to cut stellar_mass and sfr early to make norm arrays
    
    # also dividing by under mass for later graph
    stellar_mass_log = np.log10(stellar_mass[end_index:start_index])
    stellar_mass_div = stellar_mass[end_index:start_index] / under_mass
    
    mass_norm = find_norm_array(stellar_mass_log)
    
    # also dividing by under sfr for later graph
    sat_sfr = sat_sfr[end_index:start_index]
    sat_sfr_div = sat_sfr[end_index:start_index] / under_sfr
    sat_sfr_norm = find_norm_array(sat_sfr)
        
    # only keeping the last 100-start_index snapshots
    distance = distance[end_index:start_index]
    
    sat_sn = sat_sn[end_index:start_index]
    sat_cm = sat_cm[end_index:start_index]
    central_cm = central_cm[end_index:start_index]
    central_rvir = central_rvir[end_index:start_index]
    sat_mass = sat_mass[end_index:start_index]
    sat_is_primary = sat_is_primary[end_index:start_index]
    
    times_o = times_o[end_index:start_index]
        
    # interpolate masses and distances (to make plots easier to look at)
    distance_interp = interpolate(distance, times_o, start=start, end=end)
    
    mass_norm_interp = interpolate(mass_norm, times_o, start=start, end=end)
    stellar_mass_interp = interpolate(stellar_mass_div, times_o, start=start, end=end)

    central_rvir_interp = interpolate(central_rvir, times_o, start=start, end=end)
    
    sfr_norm_interp = interpolate(sat_sfr_norm, times_o, start=start, end=end)
    sat_sfr_interp = interpolate(sat_sfr, times_o, start=start, end=end)
    
    # find maxes and mins of separation
    maximums = signal.argrelextrema(distance_interp, np.greater)
    minimums = signal.argrelextrema(distance_interp, np.less)
    

    # return data (distance_interp_cut, mass_norm_interp_cut, stellar_mass_interp_cut),
    return (distance_interp, mass_norm_interp, stellar_mass_interp, sfr_norm_interp, sat_sfr_interp, central_rvir_interp), full_indices, snap, time_under

# Function for the crowded plots:

## We take in the data from find_data() and run it through here to plot it.

In [ ]:
def plot_all(start, start_index, end, end_index,
             full_data, indices, mass_or_sfr,
             bound_t, bound_ins_t, bound_exs_t, oshs_t, fuzz_t,
             full_masses, closest):
    
    # random lists or values that you might ignore
    colors_masses = ['red', 'orange', 'gold', 'green', 'blue', 'indigo', 'violet']
    
    r = np.linspace(start, end, 700)
    
    # let's setup the figure first in a pretty redundant way 
    # (setting all the axis manually because the plot i'm making is crowded)
    fig = plt.figure(figsize=(90,70))

    gs = fig.add_gridspec(7,8)
    
    ax1 = fig.add_subplot(gs[0:2, 0:5]) # distance vs time, colorbar mass, vert lines as snaps
    ax2 = fig.add_subplot(gs[2:4, 0:5]) # mass and sep over time compared
    ax3 = fig.add_subplot(gs[4:6, 0:5]) # sfr and sep over time compared 
    ax4 = fig.add_subplot(gs[6:7, 0:5]) # bound, fuzz total over time
    
    # below are all the bound, fuzz profiles on snapshots
    ax5 = fig.add_subplot(gs[0, 5:8]), 
    ax6 = fig.add_subplot(gs[1, 5:8])
    ax7 = fig.add_subplot(gs[2, 5:8]), 
    ax8 = fig.add_subplot(gs[3, 5:8]), 
    ax9 = fig.add_subplot(gs[4, 5:8]), 
    ax10 = fig.add_subplot(gs[5, 5:8]), 
    ax11 = fig.add_subplot(gs[6, 5:8]), 
    
    # to titles and axes labels and all that jazz
    
    
    #ax1
    ax1.set_title('Separation Over Time', fontsize=100)
    ax1.set_xlabel('Time / Gyr', fontsize=80)
    ax1.set_ylabel(r'Separation / $R_{vir,central}$', fontsize=80)
    
    ax1.set_xlim(end - 0.2, start + 0.2)

    #ax2
    ax2b = ax2.twinx()
    
    ax2.set_title('Mass of Sat Against Sep', fontsize=100)
    ax2.set_xlabel('Time / Gyr', fontsize=80)
    ax2.set_ylabel(r'log$_{10}$(M$_{sat}$)', fontsize=80)
    
    ax2.spines['left'].set_color('blue')
    ax2.tick_params(axis='y', colors='blue')
    ax2.yaxis.label.set_color('blue')
    
    ax2.set_xlim(end - 0.2, start + 0.2)
    
    ax2b.set_ylabel('Separation / kpc', fontsize=80)

    ax2b.spines['right'].set_color('red')
    ax2b.tick_params(axis='y', colors='red')
    ax2b.yaxis.label.set_color('red')
    
    #ax3
    ax3b = ax3.twinx()
    
    ax3.set_title('Mass of Sat Against SFR', fontsize=100)
    ax3.set_xlabel('Time / Gyr', fontsize=80)
    ax3.set_ylabel(r'log$_{10}$(M$_{sat}$)', fontsize=80)
    
    ax3.spines['left'].set_color('blue')
    ax3.tick_params(axis='y', colors='blue')
    ax3.yaxis.label.set_color('blue')
    
    ax3.set_xlim(end - 0.2, start + 0.2)
    
    ax3b.set_ylabel('SFR (M* / yr)', fontsize=80)

    ax3b.spines['right'].set_color('red')
    ax3b.tick_params(axis='y', colors='red')
    ax3b.yaxis.label.set_color('red')
    
    #ax4
    ax4.set_title('Different Masses Over Time', fontsize=100)

    ax4.set_xlabel('Time / Gyr', fontsize=80)
    ax4.set_ylabel('log10(M*)', fontsize=80)
    
    #axs5-11
    count = 0
    for i in range(5,12):
        axis = fig.axes[i-1]
        
        axis.set_title('snapshot: {}, redshift: {}'.format(snapshots[count], snap_z[count]), c=colors_masses[count], fontsize=80)

        axis.set_xlabel('R / kpc', fontsize=60)
        axis.set_ylabel(r'log$_{10}(M_{sun})$ / $\mathrm{kpc}^{3}$', fontsize=60)
        
        count += 1 
    
    # now for the plotting itself
    
    # ax1
    # average of the distance profiles
    distance_profiles = full_data[:,0]
    
    if mass_or_sfr == 'mass':
        colorbar_norm = full_data[:,1]
        colorbar_notnorm = full_data[:,2]
        
        number = 1
        
    if mass_or_sfr == 'sfr':
        colorbar_norm = full_data[:,3]
        colorbar_notnorm = full_data[:,4]
        
        number = 3
        
    # remove problem indices
    distance_redone, norm_redone, notnorm_redone = list(), list(), list()
    for i in range(len(distance_profiles)):
        d_prof, n_prof, nn_prof = distance_profiles[i], colorbar_norm[i], colorbar_notnorm[i]
        
        distance_redone.append(np.delete(d_prof, indices, axis=0))
        norm_redone.append(np.delete(n_prof, indices, axis=0))
        notnorm_redone.append(np.delete(nn_prof, indices, axis=0))
    
    av_profile = np.mean(np.array(distance_redone), axis=0)
    av_norm_profile = np.mean(np.array(norm_redone), axis=0)
    av_notnorm_profile = np.mean(np.array(notnorm_redone), axis=0)
    
    # each individual distance profile    
    for i in full_data:
        # i[0] is separation and i[5] is rvir of central
        ax1.scatter(r, i[0] / i[5] , c=cm.viridis_r(i[number]))
        
        
    # do colorbar info (depends on if mass or sfr)
    colorbar = fig.colorbar(cm.ScalarMappable(norm=Normalize(min_c,
                                     max_c), cmap='viridis_r'), ax=ax1)

    # add colorbar name depending on mass_or_sfr)
    if mass_or_sfr == 'mass':
        colorbar.set_label(r'$M_{stellar, sat}$ / $M_{stellar, 2*rvir}$', fontsize=80)
    
    if mass_or_sfr == 'sfr':
        colorbar.set_label(r'$SFR_{sat} (M_{sun, year}$ / $M_{sun, 2*rvir})$', fontsize=80)

    # add vertical lines at snapshots and when under
    for i in range(len(snap_times)):
        if end - 0.2 < snap_times[i] < start + 0.2:
            ax1.axvline(snap_times[i], c=colors_masses[i], label='z = ' + str(snap_z[i]), alpha=0.8)
            
    if time_under is not None:
        ax1.axvline(time_under, c='black', label='under', alpha=0.8)
    
    # add horizontal line at sep / rvir = 2
    ax1.axhline(y=2)
    
    # ax2
    for i in full_data:
        # i[0] is separation and i[5] is rvir of central
        ax2.scatter(r, i[2], c='blue')
        
        # separation
        ax2b.scatter(r, i[0], linestyle='dashed', c='red')
        
        
    if time_under is not None:
        ax2.axvline(time_under, c='black', label='under', alpha=0.8)
    
    
    # ax3
    for i in full_data:
        # i[0] is separation and i[5] is rvir of central
        ax3.scatter(r, i[2], c='blue')
        
        # separation
        ax3b.scatter(r, i[4], linestyle='dashed', c='red')
        
        
    if time_under is not None:
        ax3.axvline(time_under, c='black', label='under', alpha=0.8)
    
    
    # ax4
    ax4.plot(snap_times, bound_t, c='red', label='bound')
    ax4.plot(snap_times, bound_ins_t, c='green', label='insitu')
    ax4.plot(snap_times, bound_exs_t, c='blue', label='exsitu')

    ax4.axvline(time_under, c='black', label='under')
    
    
    # axs5-11
    count = 0
    for i in range(5, 12):
        axis = fig.axes[i-1]
        
        mass_types = full_masses[count]
        snapnum = snapshots[count]

        bins is mass_types[5]
        #axis.plot(mass_types[6], np.log10(mass_types[0]), c='red', label='bound')
        axis.plot(mass_types[6], np.log10(mass_types[0]), c='red')
        #axis.plot(mass_types[6], np.log10(mass_types[1]), c='green', label='insitu')
        axis.plot(mass_types[6], np.log10(mass_types[1]), c='green')
        #axis.plot(mass_types[6], np.log10(mass_types[2]), c='blue', label='exsitu')
        axis.plot(mass_types[6], np.log10(mass_types[2]), c='blue')
        #axis.plot(mass_types[6], np.log10(mass_types[4]), c='orange', label='fuzz')
        axis.plot(mass_types[6], np.log10(mass_types[4]), c='orange')

        #if snapshot == closest, add to other snapshots
        if snapnum == closest:
            for j in range(5, 12):
                axis_b = fig.axes[j-1]
                
                axis_b.plot(mass_types[6], np.log10(mass_types[0]), c='red', alpha=0.5, label='bound, ' + str(closest), linestyle='dashed')
                axis_b.plot(mass_types[6], np.log10(mass_types[1]), c='green', alpha=0.5, label='insitu, ' + str(closest), linestyle='dashed')
                axis_b.plot(mass_types[6], np.log10(mass_types[2]), c='blue', alpha=0.5, label='exsitu, ' + str(closest), linestyle='dashed')
                axis_b.plot(mass_types[6], np.log10(mass_types[4]), c='orange', alpha=0.5, label='fuzz, ' + str(closest), linestyle='dashed')
                #axis_b.plot(mass_types[6], np.log10(mass_types[0]), c='red', alpha=0.5, linestyle='dashed')
                #axis_b.plot(mass_types[6], np.log10(mass_types[1]), c='green', alpha=0.5, linestyle='dashed')
                #axis_b.plot(mass_types[6], np.log10(mass_types[2]), c='blue', alpha=0.5, linestyle='dashed')
                #axis_b.plot(mass_types[6], np.log10(mass_types[4]), c='orange', alpha=0.5, linestyle='dashed')
    
        count += 1
    
    # adding legends at the end
    for i in fig.axes:
            i.legend(loc='upper right', prop={'size': 40})
    
    fig.tight_layout()
    
    # close plots to save memory
    plt.close()
    
    return fig

# Now that I've defined all the functions, let's work on getting satellites to run the code.

## First, do a mass cut of all my satellites (log10(Mstellar,sat) > 11.4 at *any* z).

In [ ]:
# code to look at satellites and see if 
# the plan is to run through each satellite, if at some point the mass is greater than 
# our cutoff, then mark it down to add
all_l, all_central_l = list(), list()
for i in range(len(tree_SubhaloMassType)):
    if is_primary_099[i] == 1:
        central_i = i
        
    else:
        stellar_masses = np.log10(tree_SubhaloMassType[i,:,4])

        check = any(i >= 11.4 for i in stellar_masses)
        
        if check == True:
            all_l.append(i)
            all_central_l.append(central_i)

    
sat_index = all_l
central_index = all_central_l

## Second, the data may have errors where a given satellite does not have a central to go along with it. Check that they do through each file, cut the ones that don't: 

In [ ]:
# now to check if my centrals exist
# iterate through each file, where iterate through each galaxy. check if the group id of that galaxy has a host
really_catch_dict = dict()
for i in file_list:
    print('Dealing with file: {}'.format(i))

    tng_data_z = h5py.File(file_path + i, 'r')

    # find snapshot num 
    snapnum = int(i[29:31])

    match, goodmatch, idx = find_id(tng_data_099, tng_data_z, snapnum)

    # find if high or low redshift
    if i == file_list[0]:
        high_z = False
        matcher = goodmatch

    else:
        high_z = True
        matcher = match

    tng_catgrp_id = find_catgrp_id(tng_data_z, matcher, idx, high_z)

    # run through each index in sat_index to find the specific catgrp_ids need to look out for
    grp_id_lookout_list = list()
    for j in sat_index:
        if high_z == False:
            grp_id_lookout_list.append(tng_catgrp_id[match][j])

        if high_z == True:
            grp_id_lookout_list.append(tng_catgrp_id[match[j]])

    grp_ids = list(tng_data_z['catgrp_id'])
    primaries = list(tng_data_z['catgrp_is_primary'])
    gal_dict = dict()
    for j in range(len(list(grp_ids))):
        current_grp_id = grp_ids[j]
        primary = primaries[j]

        # add group id
        if current_grp_id not in list(gal_dict.keys()):
            gal_dict[current_grp_id] = 0

        # change value to 1 if is central
        if primary == 1:
            gal_dict[current_grp_id] = 1

    # check if any 0s
    keys = list(gal_dict.keys())
    catch, really_catch = list(), list()
    for j in keys:
        if gal_dict[j] == 0:
            if j in grp_id_lookout_list:
                really_catch.append(j)
            else:
                catch.append(j)

    really_catch_dict[i] = really_catch

## Third, I also only want galaxies that are satellites for a longer period of time (should see more evidence of tidal stripping):

In [ ]:
# code to find galaxies that are satellites for quite a while
consistent_sat, consistent_central = list(), list()
for i in range(len(sat_index)):
    # get trees
    sat_is_primary = tree_is_primary[sat_index[i], :]
    
    tree_tophat_s = tree_Group_M_TopHat200[sat_index[i]]
    tree_tophat_c = tree_Group_M_TopHat200[central_index[i]]
    
    # cut is_primary tree to wanted length (keeping the last 20)
    sat_is_primary = sat_is_primary[len(sat_is_primary) - 20:]
    
    # see if conforms to what want
    count = list(sat_is_primary).count(True)
    same = list(tree_tophat_s[80:] == tree_tophat_c[80:]).count(True) == 20  
    
    if count == 0 and same is True:
        consistent_sat.append(sat_index[i])
        consistent_central.append(central_index[i])

# remove certain points from the list
# centrals at 0,0 at points
# note that i did this manually
zero_index = [list(consistent_sat).index(250), list(consistent_sat).index(282), 
              list(consistent_sat).index(366), list(consistent_sat).index(424), 
              list(consistent_sat).index(429), list(consistent_sat).index(433), 
              list(consistent_sat).index(459), list(consistent_sat).index(499), 
              list(consistent_sat).index(584), list(consistent_sat).index(609),
              list(consistent_sat).index(983)]

remove_index = zero_index #+ inf_index

consistent_sat_r = np.delete(np.array(consistent_sat), remove_index)
consistent_central_r = np.delete(np.array(consistent_central), remove_index)

#choose a number of random indexes to use
number = 2

rand_indices = random.choices(list(range(len(consistent_sat_r))), k=number)

consistent_sat_r = consistent_sat_r[rand_indices]
consistent_central_r = consistent_central_r[rand_indices]

print('sat:', list(consistent_sat_r))
print('cent:', list(consistent_central_r))

# With the needed satellite cuts out of the way, let's run the code from the functions above:

In [ ]:
# code to look at individual plots of the satellite's position relative to the centrals.
# is tracked across all snapshots via tree, and colorcoded by the mass of the satellite

# 'mass' or 'sfr'
mass_or_sfr = 'mass'

# start at z=0
start_index = 99
start_time = times[start_index]

# which redshift indix to end at (indices closer to 99 start later than indices closer to 0.)
# index of 0 is the earliest snapshot we have
end_index = 50
end_time = times[end_index]


# run through each satellite and make the plot individually
for i in range(len(consistent_sat_r)):
    # show what satellite am working on   
    print('Working on index: {} / sat: {}'.format(i, consistent_sat_r))
    
    # initialize data
    full_data = list()
    full_indices = list()
    full_switch = list()
    
    max_c, min_c = np.nan, np.nan
    
    #grab which sat and central pair are looking at right now
    sat = consistent_sat_r[i]
    central = consistent_central_r[i]

    # find the data like before
    data, indices, snap_under, time_under = find_data(sat, central, 
                                              start_index=start_index, start=start_time, 
                                              end_index = end_index, end=end_time)
    
    # parse certain data
    distance = data[0]
    mass_norm = data[1]
    stellar_mass = data[2]
    sfr_norm = data[3]
    sat_sfr = data[4]
    central_rvir = data[5]
    
    # if a galaxy without the 0,0,0 error
    if data is not None: 
        full_data.append(data)
        full_indices.extend(list(indices))

        # add to max and min as needed for colorbar (hence *_c)  
        if np.isnan(max_c) and np.isnan(min_c):
            max_c = np.nanmax(stellar_mass)
            min_c = np.nanmin(stellar_mass)

        if max_c < np.nanmax(stellar_mass):
            max_c = np.nanmax(stellar_mass)

        if min_c > np.nanmin(stellar_mass):
            min_c = np.nanmin(stellar_mass)

    # remove duplicates from indices
    indices_new = [] 
    [indices_new.append(x) for x in full_indices if x not in indices_new] 

    # keep indices after the start_index
    indices_new = [i for i in indices_new if i <= start_index]

    # find where to put vertical lines (at snapshots)
    snapshots = [99, 84, 72, 67, 59, 50, 40]
    snap_times = times[snapshots]
    snap_z = redshifts[snapshots]
    
    # find which snapshot was closest to switch
    closest = min(snapshots, key=lambda x:abs(x-snap_under))
    
    # plot fig

    bound_t, bound_ins_t, bound_exs_t, oshs_t, fuzz_t = list(), list(), list(), list(), list()
    full_masses, full_distances = list(), list()
    for i in reversed(range(len(file_list))):
        # first, let's get the data ready for plotting
        
        # snapnum
        snapnum = int(file_list[i][29:31])

        redshift_z = redshifts[snapnum]

        # h5py file itself
        tng_data_z = h5py.File(file_path + file_list[i], 'r')

        # used to match galaxies across files
        match, goodmatch, idx = find_id(tng_data_099, tng_data_z, snapnum)

        # find matcher and high_z by where in file_list/if z=0 or not
        if i == 0:
            matcher = goodmatch
            high_z = False

        else:
            matcher = match
            high_z = True

        # find sfr
        sfr = find_star_formation(tng_data_z, matcher, idx, sat, high_z)

        # grab masses of galaxy
        bound, bound_insitu, bound_exsitu, oshs, fuzz, volume, bins = find_3d_masses(tng_data_z, matcher, idx, sat, high_z)
        full_masses.append((bound, bound_insitu, bound_exsitu, oshs, fuzz, volume, bins))

        distance = find_distances(tng_data_z, matcher, idx, sat, central, redshift_z, high_z)
        full_distances.append(distance)
        
        # for ax4
        bound_t.append(np.log10(sum(bound * volume)))
        bound_ins_t.append(np.log10(sum(bound_insitu * volume)))
        bound_exs_t.append(np.log10(sum(bound_exsitu * volume)))
        oshs_t.append(np.log10(sum(oshs * volume)))
        fuzz_t.append(np.log10(sum(fuzz * volume)))
        
    # run the now processed data through the plotting function
    fig = plot_all(start_time, start_index, end_time, end_index,
             np.array(full_data), np.array(indices_new), 'mass',
             bound_t, bound_ins_t, bound_exs_t, oshs_t, fuzz_t,
             full_masses, closest)
    
    # save to whichever folder and with whichever filename you want
    fig.savefig('sat_{}.png'.format(sat), bbox_inches='tight')

# It's also helpful to look at the 2D profiles of galaxies (had been doing 3D). The classes and functions used were written by Song (like GalaxyMap, aper_summary(), etc.)

## This is how you look at the 2D profile for a specific galaxy using his code.

In [ ]:
# code to look at the profiles of the galaxies using crusher (another check)
tng_data = BeneMassAgeZMaps(file_path + file_list[0], label='tng100')

total = list()
for i in range(len(consistent_sat_r)):
    print('Dealing with galaxy {} out of {}...'.format(i+1, len(consistent_sat_r)))
    
    gal = GalaxyMap(tng_data, consistent_sat_r[i], proj='xy')

    gal.aper_summary()
    gal.ell_summary()
    
    prof = visual.prepare_show_ellipse(gal.info, gal.maps, gal.ell_sum)
    
    mass_profile = prof['ell_gal_3']['intens']
    radius = prof['ell_gal_3']['r_kpc']
    
    total.append((mass_profile, radius))

In [ ]:
# As a check, I was comparing the subhalo velocity, star formation rates, and velocity distributions:

In [ ]:
field_l = ['tree_SubhaloVel', 'tree_SubhaloSFR', 'tree_SubhaloVelDisp']
number = 0
gal = 33

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()

ax1.plot(np.abs(tng_data_099[field_l[number]][gal,:,0]) + np.abs(tng_data_099[field_l[number]][gal,:,1]) + np.abs(tng_data_099[field_l[number]][gal,:,2]))
ax2.scatter(np.linspace(0,99, len(tng_data_099['tree_is_primary'][gal])), tng_data_099['tree_is_primary'][gal], s=1, c='r', alpha=0.6)

ax1.set_ylabel(field_l[number], c='b')
ax2.set_ylabel('is_primary', c='r')

# Below is just a previous list of satellites and centrals I was using. I don't think you need to pay this much heed?

In [ ]:
"""
previous sats/centrals (20 without central check):

sat: [3, 4, 5, 7, 10, 11, 16, 17, 19, 20, 21, 23, 25, 29, 43, 46, 49, 54, 57, 59, 60, 69, 90, 92, 107, 111, 117, 118, 124, 126, 136, 138, 140, 145, 146, 147, 150, 151, 156, 157, 161, 163, 164, 169, 171, 172, 174, 176, 179, 185, 187, 189, 191, 192, 194, 197, 201, 203, 208, 211, 214, 215, 237, 245, 246, 252, 254, 269, 270, 279, 288, 289, 293, 294, 295, 299, 302, 306, 327, 342, 354, 359, 361, 364, 371, 374, 376, 377, 393, 408, 409, 427, 440, 442, 444, 447, 449, 465, 472, 504, 508, 516, 536, 546, 558, 587, 605, 626, 627, 632, 635, 642, 677, 682, 683, 699, 703, 724, 748, 759, 792, 805, 816, 848, 862, 894, 900, 922, 970, 1073, 1209, 1211, 1281, 1283, 1371, 1421, 1455, 1673]
cent: [0, 0, 0, 0, 0, 0, 12, 12, 12, 12, 12, 22, 22, 22, 41, 41, 47, 51, 51, 51, 51, 62, 89, 89, 104, 108, 115, 115, 123, 123, 132, 132, 139, 142, 142, 142, 148, 148, 154, 154, 160, 160, 160, 165, 170, 170, 170, 175, 175, 184, 184, 184, 190, 190, 193, 195, 198, 198, 207, 210, 213, 213, 232, 244, 244, 251, 251, 267, 267, 277, 287, 287, 291, 291, 291, 298, 301, 304, 326, 341, 353, 358, 360, 363, 370, 372, 375, 375, 392, 406, 406, 426, 437, 441, 443, 446, 448, 464, 471, 503, 506, 515, 535, 544, 557, 586, 603, 625, 625, 631, 633, 640, 676, 681, 681, 698, 702, 723, 747, 758, 791, 804, 814, 846, 861, 893, 899, 921, 969, 1072, 1208, 1210, 1280, 1282, 1370, 1420, 1454, 1672]
""";